## Зареждане на данните

Зареждаме данните от sklearn. Ще използваме `train` set-a.

In [1]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')

Приготвяме си удобен `DataFrame` с уникално `doc_id` за всяка статия, както и нейния `text` и `topic`.

In [2]:
import pandas as pd

df = pd.DataFrame({
    'doc_id': list(map(lambda filename: '/'.join(filename.split('/')[-2:]), newsgroups_train.filenames)),
    'topic': list(map(lambda topic_id: newsgroups_train.target_names[topic_id], newsgroups_train.target)),
    'text': newsgroups_train.data,
})

df = df.set_index('doc_id')

df

,topic,text
doc_id,,
rec.autos/102994,rec.autos,From: lerxst@wam.umd.edu (where's my thing)\nS...
comp.sys.mac.hardware/51861,comp.sys.mac.hardware,From: guykuo@carson.u.washington.edu (Guy Kuo)...
comp.sys.mac.hardware/51879,comp.sys.mac.hardware,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
comp.graphics/38242,comp.graphics,From: jgreen@amber (Joe Green)\nSubject: Re: W...
sci.space/60880,sci.space,From: jcm@head-cfa.harvard.edu (Jonathan McDow...
talk.politics.guns/54525,talk.politics.guns,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...
sci.med/58080,sci.med,From: bmdelane@quads.uchicago.edu (brian manni...
comp.sys.ibm.pc.hardware/60249,comp.sys.ibm.pc.hardware,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...
comp.os.ms-windows.misc/10008,comp.os.ms-windows.misc,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...


## Примерен текст

In [3]:
print(df.iloc[0]['text'])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







## Почистване на данните

Текстовете биват преработени по следните начини:
- токенизираме с `TweetTokenizer`, който разделя текста на отделни думи
- правим всички думи да са с малка буква
- взимаме само думи и махаме всички други специални символи, които зашумяват драстично речника
- махаме стоп думите, за да не зашумяват векторите от векторното пространство
- прилагаме `stemming`, за да намалим още речника на векторите

## Още идеи за почистване на данните

Почти всички текстове имат няколко реда от вида:

```
From: lerxst@wam.umd.edu (where's my thing)
Nntp-Posting-Host: rac3.wam.umd.edu
```

Думи като `rac3`, `wam`, `umd`, `lerxst` са части от интернет имена или домейни, които не носят никакъв смисъл и зашумяват речника на векторното пространство и могат да бъдат отстранени като проверим дали дадената дума присъства в английския речник.

In [4]:
from nltk.tokenize import TweetTokenizer
tweet_tok = TweetTokenizer()

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def clean_text(text):
    # TODO: remove words, which do not exist in English dictionary
    # TODO: Rake
    return [
        stemmer.stem(word.lower())
        for word in tweet_tok.tokenize(text)
        if word.isalpha() and not word in stop_words
    ]

In [5]:
sample = list(df['text'].head(2))
list(map(clean_text, sample))

[['from',
  'thing',
  'subject',
  'what',
  'car',
  'organ',
  'univers',
  'maryland',
  'colleg',
  'park',
  'line',
  'i',
  'wonder',
  'anyon',
  'could',
  'enlighten',
  'car',
  'i',
  'saw',
  'day',
  'it',
  'door',
  'sport',
  'car',
  'look',
  'late',
  'earli',
  'it',
  'call',
  'bricklin',
  'the',
  'door',
  'realli',
  'small',
  'in',
  'addit',
  'front',
  'bumper',
  'separ',
  'rest',
  'bodi',
  'thi',
  'i',
  'know',
  'if',
  'anyon',
  'tellm',
  'model',
  'name',
  'engin',
  'spec',
  'year',
  'product',
  'car',
  'made',
  'histori',
  'whatev',
  'info',
  'funki',
  'look',
  'car',
  'pleas',
  'thank',
  'il',
  'brought',
  'neighborhood',
  'lerxst'],
 ['from',
  'guy',
  'kuo',
  'subject',
  'si',
  'clock',
  'poll',
  'final',
  'call',
  'summari',
  'final',
  'call',
  'si',
  'clock',
  'report',
  'keyword',
  'si',
  'acceler',
  'clock',
  'upgrad',
  'd',
  'shelley',
  'organ',
  'univers',
  'washington',
  'line',
  'a',
  

В следващата клетка почистваме текстовете и образумва последователност от думи, които после подаваме на `TfidfVectorizer`-а, за да създаде векторното пространство.

In [6]:
df['text_clean'] = df['text'].apply(clean_text).apply(lambda x: ' '.join(x))
df

,topic,text,text_clean
doc_id,,,
rec.autos/102994,rec.autos,From: lerxst@wam.umd.edu (where's my thing)\nS...,from thing subject what car organ univers mary...
comp.sys.mac.hardware/51861,comp.sys.mac.hardware,From: guykuo@carson.u.washington.edu (Guy Kuo)...,from guy kuo subject si clock poll final call ...
comp.sys.mac.hardware/51879,comp.sys.mac.hardware,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,from thoma e willi subject pb question organ p...
comp.graphics/38242,comp.graphics,From: jgreen@amber (Joe Green)\nSubject: Re: W...,from jgreen joe green subject re weitek organ ...
sci.space/60880,sci.space,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,from jonathan mcdowel subject re shuttl launch...
talk.politics.guns/54525,talk.politics.guns,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,from foxvog dougla subject re reword second am...
sci.med/58080,sci.med,From: bmdelane@quads.uchicago.edu (brian manni...,from brian man delaney subject brain tumor tre...
comp.sys.ibm.pc.hardware/60249,comp.sys.ibm.pc.hardware,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,from grubb subject re ide vs scsi organ new me...
comp.os.ms-windows.misc/10008,comp.os.ms-windows.misc,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,from subject win icon help pleas organ univers...


## Векторизиране на текстовете с TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text_clean'])

print('Shape of the vector matrix: {}'.format(X.shape))

## prints the vocabulary of the vector space
# print(vectorizer.get_feature_names())

Shape of the vector matrix: (11314, 56567)


### Понеже TF-IDF векторите са много големи (разредени), прилагаме LSA (Latent Semantic Analysis) чрез SVD (Singular Vector Decomposition) в 100 измерения

In [8]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, n_iter=5, random_state=1)
svd.fit(X)

X_reduced = svd.transform(X)

print('Shape of the vector matrix after LSA (SVD): {}'.format(X_reduced.shape))

Shape of the vector matrix after LSA (SVD): (11314, 100)


## Изчисляване на Косинусова сходимост между всеки 2 текста и съхраняването на сходимостите като матрица

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X, X)

similarity_matrix_lsa = cosine_similarity(X_reduced, X_reduced)

## Препоръчване на Top N текстове, спрямо 1 вече прочетен от потребителя текст

За целта се използва матрицата на Косинусови сходимости.
Намираме редът на статията, която е прочел потребителят и сортираме всички колони в намаляващ ред.
Тоест искаме да намерим кои са най-сходните статии с тази, която е чел потребителят.
Винаги на първа позиция ще имаме максимална сходимост 1, защото диагоналът на матрицата е само единици, защото всяка статия е напълно сходна със самата себе си.
Затова взимаме `top_n` статии от втората включително нататък.
Накрая връщаме `DataFrame` с препоръчаните текстове в намаляващ ред, както и техните коефициенти на сходимост.

Коефициентите на сходимост са много полезни за проверка дали модификации по алгоритъма ни или преработката на данните увеличават сходимостта или я намаляват.

In [10]:
import numpy as np

def recommend_articles(doc_id, top_n=10, similarity_matrix=similarity_matrix):
    indexed_doc_ids = pd.Series(df.index)

    matched_row_number = indexed_doc_ids[indexed_doc_ids == doc_id].index[0]

    matched_row = pd.Series(similarity_matrix[matched_row_number])

    sorted_recommendations = matched_row.sort_values(ascending=False)

    top_n_recommendations = sorted_recommendations[1:(top_n + 1)]
    
    return pd.DataFrame({
        'doc_id': indexed_doc_ids[top_n_recommendations.index].values,
        'similarity': top_n_recommendations.values
    }).set_index('doc_id')

In [11]:
recommend_articles('comp.sys.mac.hardware/51861', top_n=20)

,similarity
doc_id,
comp.sys.mac.hardware/51695,0.648966
comp.sys.mac.hardware/51674,0.440609
comp.sys.mac.hardware/51560,0.433631
comp.sys.mac.hardware/51920,0.374815
comp.sys.mac.hardware/51642,0.273637
comp.sys.mac.hardware/51906,0.269982
comp.sys.mac.hardware/51708,0.247467
comp.sys.mac.hardware/51747,0.214782
comp.sys.mac.hardware/51745,0.195968


In [15]:
recommend_articles('comp.sys.mac.hardware/51861', top_n=20, similarity_matrix=similarity_matrix_lsa)

,similarity
doc_id,
comp.sys.mac.hardware/51695,0.936158
comp.sys.mac.hardware/51560,0.854279
comp.sys.mac.hardware/51674,0.791534
comp.sys.mac.hardware/51745,0.772657
comp.sys.mac.hardware/51636,0.768200
comp.sys.mac.hardware/51507,0.754584
comp.sys.mac.hardware/51747,0.744023
comp.sys.mac.hardware/51708,0.737015
comp.sys.mac.hardware/51895,0.718575


## Препоръчване на статии спрямо няколко прочетени статии от потребителя (т.е. потребителски профил)

In [20]:
from functools import reduce

def recommend_articles_many(doc_ids, top_n=50, sim_matrix=similarity_matrix):
    all_recommendations = [recommend_articles(doc_id, similarity_matrix=sim_matrix) for doc_id in doc_ids]
    combined = reduce(lambda a, b: a.append(b), all_recommendations)
    return combined.sort_values(by=['similarity'], ascending=False).head(top_n)

### Потребителският профил е просто набор от няколко статии

В случая, първите 10 или 100 oт всички статии.

In [24]:
user_profile = df.index[:10]
user_profile

Index(['rec.autos/102994', 'comp.sys.mac.hardware/51861',
       'comp.sys.mac.hardware/51879', 'comp.graphics/38242', 'sci.space/60880',
       'talk.politics.guns/54525', 'sci.med/58080',
       'comp.sys.ibm.pc.hardware/60249', 'comp.os.ms-windows.misc/10008',
       'comp.sys.mac.hardware/50502'],
      dtype='object', name='doc_id')

## Резултати с прости TF-IDF вектори

In [21]:
recommend_articles_many(doc_ids=user_profile, top_n=50)

,similarity
doc_id,
talk.politics.guns/54376,0.953281
comp.sys.ibm.pc.hardware/60247,0.910806
comp.sys.ibm.pc.hardware/60381,0.899665
comp.sys.ibm.pc.hardware/60392,0.859549
comp.sys.ibm.pc.hardware/60294,0.858942
comp.sys.ibm.pc.hardware/60435,0.832505
comp.sys.ibm.pc.hardware/60143,0.794569
comp.sys.ibm.pc.hardware/60888,0.785147
comp.sys.ibm.pc.hardware/60196,0.736074


## Резултати с редуцираните вектори след LSA са с далеч по-голяма сходимост към потребителския профил

In [23]:
recommend_articles_many(doc_ids=user_profile, top_n=50, sim_matrix=similarity_matrix_lsa)

,similarity
doc_id,
comp.sys.ibm.pc.hardware/60247,0.993705
comp.sys.ibm.pc.hardware/60381,0.990689
comp.sys.ibm.pc.hardware/60392,0.990321
comp.sys.ibm.pc.hardware/60294,0.986806
talk.politics.guns/54376,0.983535
comp.sys.ibm.pc.hardware/60888,0.981674
comp.sys.ibm.pc.hardware/60435,0.974264
comp.sys.ibm.pc.hardware/60383,0.968329
comp.sys.ibm.pc.hardware/60143,0.960013


## Oще идеи за подобрение

Аз лично бих искал да видя какви резултати ще получим, ако вместо TF-IDF използваме **word embeddings** (пр. GloVe или word2vec). Все пак търсим съдържание, което семантично е най-близко до съдържанието, което потребителят е харесал.

Как бихме приложили **word embeddings**? Можем да вземем думите от текста (след почистването) и да осредним векторите на всички думи, за да получим "средния смисъл" на статията. След това отново можем да сравним всяка статия с всяка друга по "осреднен смисъл" чрез косинусова сходимост.